In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle
import statistics
import community
import matplotlib as mlp
# mlp.use("Qt5Agg")
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx

import analysis as an
import pandas as pd


import numpy as np
import seaborn as sns

from scipy import stats 
import scipy
import pprint as pp
import bct as bct

In [2]:
basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'


In [ ]:
#making a dataframe
_dfs=[]
for group, data in subedge_dict.items():
    print(group)
    for sub, dat in data['graphs'].items():
        print(sub)
        tmp = pd.DataFrame.from_dict(dict(dat.nodes(data =True)), orient='index')
        tmp['subject'] = sub
        tmp['group'] = group
        tmp['IC'] = tmp.index
#         tmp['IC']='IC_%i'%tmp['IC']
        tmp['IC'] = 'IC_' + tmp['IC'].astype(str)
        _dfs.append(tmp)
total = pd.concat(_dfs)
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'demo*')
demo_dict=an.onetoughjar(latest_file)
demo_df = pd.DataFrame.from_dict(demo_dict['NR'], orient='index')
demo_df['subject'] = demo_df.index
demo_df.reset_index(inplace = True)
demo_df["subject"]=pd.to_numeric(demo_df["subject"])
total["subject"]=pd.to_numeric(total["subject"])
complete_df=pd.merge(demo_df, total, on="subject")
#complete_df.to_csv(os.path.join(basepath,'tmp','complete_data.csv'), sep=',')
#an.adillyofapickle('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets',complete_df,'11_complete_df')


In [ ]:
def zscore(col):
    col_z = (col - col.mean())/col.std(ddof=0)
    return(col_z)

In [ ]:
p='/Users/gracer/Google Drive/HCP/HCP_graph/1200/'
atlas=pd.read_csv(os.path.join(p,'brains','atlas.csv'), sep=',')


In [ ]:
labels = pd.read_csv('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/mod_labels.csv', sep=',')

In [4]:
# this is a dataframe
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'6_*')
submod_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/6_submod_dict_09-15-2020_03-49-48


In [5]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'5_*')
summary_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/5_summary_dict_09-15-2020_03-49-44


In [6]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'7_*')
subgraph_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/7_subgraph_dict_09-17-2020_08-55-55


In [7]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'10_*')
subedge_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/10_subedge_dict_09-17-2020_09-43-31


In [ ]:
subedge_dict['no'].keys()

In [ ]:
test = dict(subedge_dict['no']['graphs']['138837'].nodes(data=True))

In [ ]:
pd.DataFrame.from_dict(test, orient='index')

In [ ]:
_dfs=[]
for group, data in subedge_dict.items():
    print(group)
    for sub, dat in data['graphs'].items():
        print(sub)
        tmp = pd.DataFrame.from_dict(test, orient='index')
        tmp['subject'] = sub
        tmp['group'] = group
        tmp['IC'] = tmp.index
        tmp['IC']=tmp['IC']+1
        _dfs.append(tmp)
total = pd.concat(_dfs)

In [ ]:
total

In [ ]:
demo_dict['NR'].keys()

In [ ]:
demo_df = pd.DataFrame.from_dict(demo_dict['NR'], orient='index')

In [ ]:
demo_df['subject'] = demo_df.index
demo_df.reset_index(inplace = True)

In [ ]:
demo_df

In [ ]:
demo_df["subject"]=pd.to_numeric(demo_df["subject"])
total["subject"]=pd.to_numeric(total["subject"])

In [ ]:
pd.merge(demo_df, total, on="subject")

In [ ]:
x

In [ ]:
part = pd.DataFrame.from_dict(summary_dict['no']['modules']['partition'], orient='index', columns = ['module'])


Making an edge data frame

In [ ]:
edges = {}

for group, stuff in summary_dict.items():
    print(group)
    _df = nx.to_pandas_edgelist(stuff['comm_graph'])
    _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

    _df['group']=group
    edges[group]=_df

edge_df=pd.concat(list(edges.values()))

In [ ]:
edge_df['z_weight']=an.zscore(edge_df['weight'])

# Set normalized edge as an get_edge_attributes
for k,v in update_dict.items():
    test=edge_df[edge_df['group']==k]
    keyz = list(zip(test['source'],test['target']))
    values=test['z_weight']
    up_dict={}
    for i in range(len(keyz)):
        up_dict[keyz[i]]={'z_edge':values[i]}
    nx.set_edge_attributes(v['comm_graph'], up_dict)


In [ ]:
edge_df

Checking the save dict to calculcate differences in edges 

In [9]:
# this is a dataframe
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'3_*')
save_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/3_save_dict_08-31-2020_09-48-50


In [ ]:
for group, values in save_dict['NR'].items():
    print(group)
    values['modules'] = summary_dict[group]['modules']['partition']
    for sub, graph in values['graphs'].items():
        print(sub)
        G = graph
        modules = summary_dict[group]['modules']['partition']
        nx.set_node_attributes(G, modules, 'modules')

In [ ]:
save_dict['NR']['no'].keys()

In [ ]:
edges = {}

for group, stuff in save_dict['NR'].items():
    print(group)
    for sub, graph in stuff['graphs'].items():
        print(sub)
        _df = nx.to_pandas_edgelist(graph)
        _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

        _df['group']=group
        edges[group]=_df

sub_edge_df=pd.concat(list(edges.values()))

In [ ]:
sub_edge_df = sub_edge_df[sub_edge_df['source'] != sub_edge_df['target']]

In [ ]:
#sub_edge_df.to_csv(os.path.join(basepath,'tmp','sub_edge_data.csv'), sep=',')

In [ ]:
sub_edge_df['z_weight']=an.zscore(sub_edge_df['weight'])

In [ ]:
# Set normalized edge as an get_edge_attributes
for group,v in save_dict['NR'].items():
    for sub, graph in v['graphs'].items():
        test=sub_edge_df[sub_edge_df['group']==group]
        keyz = list(zip(test['source'],test['target']))
        values=test['z_weight']
        up_dict={}
        for i in range(len(keyz)):
            up_dict[keyz[i]]={'z_edge':values[i]}
        nx.set_edge_attributes(graph, up_dict)


Make individual comm graphs

In [ ]:
save_dict['NR']['no']['comm_graph']['200513'].edges(data=True)

In [ ]:
# Make community graph
for group,v in save_dict['NR'].items():
    print(group)
    v.update(comm_graph = {})
    for sub, data in v['graphs'].items():
        print(sub)
        comm_graph = community.induced_graph(v['modules'], data)
        v['comm_graph'][sub] = comm_graph


In [ ]:
def cal_edges(dic, basepath, group):
    edges = {}
    for sub, graph in dic.items():
        print(sub)
        _df = nx.to_pandas_edgelist(graph)
        _df.loc[(_df['source'] == _df['target']), 'weight'] = 0
        _df = _df[_df['source'] != _df['target']]
        _df['group']=group
        _df['z_weight']=an.zscore(_df['weight'])
        edges[sub]=_df
    return(edges)
    
    


    

In [ ]:
save_dict['NR']['no']['comm_graph']['157437']

In [ ]:
tmp_dict = {'no':{}, 'ov': {}, 'ob': {}}
for group, dat in save_dict['NR'].items():
    print(group)
    x=cal_edges(dat['comm_graph'], basepath, group)
    tmp_dict[group]=pd.concat(list(x.values()))


In [ ]:
tmp_dict['ob']

In [ ]:
sub_comm_edge_df=pd.concat(list(tmp_dict.values()))


In [ ]:
#sub_comm_edge_df.to_csv(os.path.join(basepath,'tmp','sub_comm_edge_data.csv'), sep=',')

In [ ]:
edges = {}

for group, stuff in save_dict['NR'].items():
    print(group)
    for sub, graph in stuff['graphs'].items():
        print(sub)
        _df = nx.to_pandas_edgelist(graph)
        _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

        _df['group']=group
        edges[group]=_df

sub_edge_df=pd.concat(list(edges.values()))
sub_edge_df = sub_edge_df[sub_edge_df['source'] != sub_edge_df['target']]
# sub_edge_df['z_weight']=an.zscore(sub_edge_df['weight'])
# sub_edge_df.to_csv(os.path.join(basepath,'tmp','sub_edge_data.csv'), sep=',')


In [ ]:
# Set normalized edge as an get_edge_attributes
for group,v in save_dict['NR'].items():
    for sub, graph in v['graphs'].items():
        test=sub_edge_df[sub_edge_df['group']==group]
        keyz = list(zip(test['source'],test['target']))
        values=test['z_weight']
        up_dict={}
        for i in range(len(keyz)):
            up_dict[keyz[i]]={'z_edge':values[i]}
        nx.set_edge_attributes(graph, up_dict)


In [ ]:
#an.adillyofapickle('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets',save_dict['NR'],'10_subedge_dict')

In [ ]:
def mod_loc(group, module): 
    columns=['Index','area', 'label']
    data = []
    part = pd.DataFrame.from_dict(summary_dict[group]['modules']['partition'], orient='index', columns = ['module'])
    x = part.loc[part['module'] == module].index
    for item in x:
        data.append(labels[['Index','area', 'label']].loc[labels['Index'][item]])    
    df = pd.DataFrame(data, columns=columns)
    return(df)

In [ ]:
mod_loc('no', 0)

# Graphs! Figures!

In [10]:
aes_dict={'no':{},
          'ov':{},
          'ob':{}}
for group, stuff in summary_dict.items():
    print(group)
    G=stuff['comm_graph']
    aes_dict[group]=an.aesthetics(G,15000,100, 'sans-serif', 'Bold', 'z_edge', (80,50), 1)

no
8
ov
9
ob
9


In [ ]:
def module_fig(G, Type, basepath, aes):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())

    positions=nx.circular_layout(G)
    plt.figure(figsize= aes['general']['plot_size'])
    ### NODES ####
    print(len(G.nodes()))
    y=nx.draw_networkx_nodes(
        G,positions,
        node_color = aes['nodes']['color'],
        node_size = aes['nodes']['node_size'],
        alpha=1.0,
        cmap= plt.get_cmap('Set3', aes['nodes']['n_color']),
        vmin=aes['nodes']['min'],
        vmax=aes['nodes']['max'])

    #Styling for node labels
    nx.draw_networkx_labels(G,
                            positions,
                            font_size = aes['nodes']['font_size'],
                            font_family= aes['nodes']['font_family'],
                            fontweight = aes['nodes']['font_weight'])
    #Node color bar stuff
    n_color = aes['nodes']['n_color']
    print(n_color)
    node_bar=plt.colorbar(y, label='Module value')
    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    print(tick_locs)
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))
    node_bar.set_label('Modularity', fontsize =  aes['nodes']['font_size'])

    ## EDGES ##
    x=nx.draw_networkx_edges(G,
                             positions,
                             edge_list=edges,
                             style='solid',
                             width = weights*aes['edges']['width mod'],
                             edge_color = weights,
                             edge_vmin=aes['edges']['min'],
                             edge_vmax=aes['edges']['max'],
                             edge_cmap= aes['edges']['colormap'])
    # Edge color bar stuff
    sm = plt.cm.ScalarMappable(cmap=aes['edges']['colormap'],
                               norm=plt.Normalize(vmin = aes['edges']['min'], vmax=aes['edges']['max']))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(aes['nodes']['font_size'])
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(aes['nodes']['font_size'])
        l.set_verticalalignment('center')

    edge_bar.set_label('Strength of edge weight', fontsize = aes['nodes']['font_size'])
    # Title things
    plt.title("Module Connectivity Weights %s"%Type, fontsize = aes['nodes']['font_size']*2)
    plt.axis('off')
    basepath= os.path.join(basepath,'images')
    plt.savefig(os.path.join(basepath,"modularity_%s.png"%(Type)), format="PNG")
    # plt.show()

In [15]:
summary_dict['no']['comm_graph'].nodes(data=True)

NodeDataView({0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}})

In [11]:
for key, graph in summary_dict.items():
    print(key)
    G=graph['comm_graph']
    aes=aes_dict[key]
    an.module_fig(G,'%s z-score'%key, basepath, aes)

no
8
8
[0.4375 1.3125 2.1875 3.0625 3.9375 4.8125 5.6875 6.5625]
ov
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]
ob
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]


In [ ]:
groups = ['no','ov','ob']
mods = [2,4,0,5,2,6]
IC_=[11,16, 27, 35, 54, 58, 85]
for group in groups:
    for mod in mods:
        for x in IC_:
            if x in list(subgraph_dict[group][mod].nodes()):
                print('for %s in mod %i and node %i'%(group, mod, x))
                print(subgraph_dict[group][mod].nodes(data=True)[x])

In [ ]:
def aesthetics(graph, node_size, font_size, font_family, font_weight, edge_att, plot_size, mod):
    aes={'general':{},
         'nodes':{},
         'edges':{}}
    # nodes
    color = np.array(list(graph.nodes))
    color = np.array(color)
    n_color=len(list(set(color)))
    print(n_color)
    aes['nodes']['color'] = color
    aes['nodes']['colormap'] = ['Set3', n_color]
    aes['nodes']['n_color'] = n_color
    aes['nodes']['max'] = float(color.max())
    aes['nodes']['min'] = float(color.min())
    aes['nodes']['font_size'] = int(font_size)
    aes['nodes']['font_family'] = font_family
    aes['nodes']['font_weight'] = font_weight
    aes['nodes']['node_size'] = int(node_size)
    #edges
    aes['edges']['colormap'] = plt.cm.gist_rainbow
    edges,weights = zip(*nx.get_edge_attributes(graph,edge_att).items())
    weights=np.array(weights)
    aes['edges']['width mod'] = mod
    aes['edges']['min'] = weights.min()
    aes['edges']['max'] = weights.max()
    # General
    aes['general']['plot_size']=plot_size #tuple 80,50
    return(aes)

In [13]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def basic_graph(graph, scl, FONT, metric, size, tile, basepath, bmi, mod):
    #threshold
    e,w = zip(*nx.get_edge_attributes(graph, 'weight').items())
    purr=np.percentile(w, tile)
    g=an.threshold2(graph,purr)
    # style
    fig = plt.figure(figsize = size)
    ax1 = plt.subplot(111, aspect = 'equal')
    plt.subplots_adjust(left=0.25, bottom=0, right=0.75, top=1, wspace=0.2, hspace=0)

    positions=nx.circular_layout(g)
    
    # NODES
    nodes, color = zip(*nx.get_node_attributes(g, metric).items())
    nodes, label = zip(*nx.get_node_attributes(g, 'labels').items())
    nodes, size = zip(*nx.get_node_attributes(g, metric).items())
    new={}
    for x,y in nx.get_node_attributes(g, 'labels').items():
        new[x]=y['area']
    
    #draws nodes
    color = np.array(color)
    colz=stats.zscore(color)
    nColormap=plt.cm.cool 
    cM=colz.max()
    cm=colz.min()

    scale=scl/colz.max()
    nx.draw_networkx_nodes(g,
                             positions,
                             node_color=colz,
                             node_size=np.square(colz)*scale,
                             alpha=0.8,
                             cmap= nColormap,
                             vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(g, 
                            positions,
                            new,
                            font_size=FONT,
                            font_family='sans-serif',
                            fontweight = 'bold',
                            horizontalalignment = 'center',
                            verticalalignment = 'top')


    # EDGES 
    edges,weights = zip(*nx.get_edge_attributes(g, 'weight').items())
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html

    nx.draw_networkx_edges(g, 
                             positions,
                             edge_list=edges,
                             style='solid',
                             width = np.square(weights)*200,
                             edge_color = weights,
                             edge_cmap=eColormap,
                             edge_vmin=weights.min(),
                             edge_vmax=weights.max())
    
    #FInishing touches


    plt.axis('off')
    if bmi == 'no':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'Average BMI'), fontsize = FONT)
    if bmi == 'ov':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'High BMI'), fontsize = FONT)
    if bmi == 'ob':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'Very high BMI'), fontsize = FONT)
    

    axins1 = inset_axes(ax1,
                    width="2.5%",  # width = 50% of parent_bbox width
                    height="100%",  # height : 5%
                    loc='right',
                    bbox_to_anchor=(0.15, 0., 1, 1), #(x0, y0, width, height)
                    bbox_transform=ax1.transAxes,
                    borderpad=1)
    
    axins2 = inset_axes(ax1,
                    width="2.5%",  # width = 50% of parent_bbox width
                    height="100%",  # height : 5%
                    loc='right',
                    bbox_to_anchor=(.25, 0, 1, 1),
                    bbox_transform=ax1.transAxes,
                    borderpad=1)

    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = weights.min(), vmax=weights.max()))
    sm._A = []

    edge_bar=fig.colorbar(sm, cax=axins1, orientation="vertical")
    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(FONT)
    edge_bar.set_label('Strength of edge weight', fontsize = FONT)
    
    sn = plt.cm.ScalarMappable(cmap=nColormap, norm=plt.Normalize(vmin = cm, vmax=cM))
    sn._A = []
    
    rgb = sn.to_rgba(x=colz)
    save={}
    i = 0
    for x,y in nx.get_node_attributes(g, 'labels').items():
        save[x]=[y['IC'], y['area'], np.round(rgb[i]*255)]
        i=i+1
    
    node_bar=fig.colorbar(sn, cax=axins2, label='Module value')
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(FONT)
    node_bar.set_label('%s'%metric,fontsize = FONT)
    
    
    basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/images'
    if bmi == 'no':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'Average BMI', mod)), format="PNG")
    if bmi == 'ov':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'High BMI', mod)), format="PNG")
    if bmi == 'ob':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'Very high BMI', mod)), format="PNG")
    plt.show()
    return(save)


participation coefficient 

In [28]:
wb_con_PC = {
    'no':{2:{}, 5:{}, 0:{}},
    'ov': {2:{}, 6:{}, 0:{}},
    'ob':{4:{}, 6:{}, 0:{}}
}

for group, mods in wb_con_PC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_PC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'PC', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
2
5
0
ov
2
6
0
ob
4
6
0


clustering coefficient 

In [29]:
wb_con_CC = {
    'no':{2:{},5:{}},
    'ov': {2:{}, 6:{}},
    'ob':{4:{}, 6:{}}
}

for group, mods in wb_con_CC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_CC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'clustering', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
2
5
ov
2
6
ob
4
6


Betweenness centrality

In [30]:
wb_con_BC = {
    'no':{0:{}},
    'ov': {0:{}},
    'ob':{0:{}}
}

for group, mods in wb_con_BC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_BC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'centrality', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
0
ov
0
ob
0


In [31]:
wb_con = {'CC':wb_con_CC, 'PC':wb_con_PC, 'BC':wb_con_BC}

## Get the colors for workbench

In [ ]:
pth='/Users/gracer/Google Drive/HCP/HCP_graph/1200/brains/color_brain'

In [33]:
for measure, _con in wb_con.items():
    print(measure)
    for group, mods in _con.items():
        print(group)
        for mod, data in mods.items():
            print(mod)
            writepath = os.path.join(pth,'%s_%s_%i.txt'%(measure, group, mod))
            f = open(writepath, "w")
            for key, value in data.items():
                print(key)
                x = value[2]
                print(x)
                f.write('LABEL_%i\n'%(key+1))
                f.write('%i %i %i %i %i\n'%(key+1, x[0], x[1], x[2], x[3]))
            f.close()

CC
no
2
2
[103. 152. 255. 255.]
3
[217.  38. 255. 255.]
7
[161.  94. 255. 255.]
10
[182.  73. 255. 255.]
12
[186.  69. 255. 255.]
15
[176.  79. 255. 255.]
16
[149. 106. 255. 255.]
17
[ 92. 163. 255. 255.]
18
[ 19. 236. 255. 255.]
19
[120. 135. 255. 255.]
20
[193.  62. 255. 255.]
22
[ 97. 158. 255. 255.]
26
[124. 131. 255. 255.]
27
[255.   0. 255. 255.]
29
[193.  62. 255. 255.]
33
[ 94. 161. 255. 255.]
34
[ 95. 160. 255. 255.]
37
[146. 109. 255. 255.]
40
[189.  66. 255. 255.]
42
[187.  68. 255. 255.]
44
[206.  49. 255. 255.]
46
[192.  63. 255. 255.]
80
[  0. 255. 255. 255.]
87
[ 75. 180. 255. 255.]
95
[160.  95. 255. 255.]
96
[ 29. 226. 255. 255.]
5
64
[200.  55. 255. 255.]
66
[ 49. 206. 255. 255.]
67
[109. 146. 255. 255.]
68
[170.  85. 255. 255.]
98
[169.  86. 255. 255.]
99
[182.  73. 255. 255.]
75
[  0. 255. 255. 255.]
78
[255.   0. 255. 255.]
48
[ 40. 215. 255. 255.]
86
[ 22. 233. 255. 255.]
62
[118. 137. 255. 255.]
ov
2
2
[118. 137. 255. 255.]
3
[229.  26. 255. 255.]
7
[179.  76. 25

In [45]:
import subprocess
import glob
for x in glob.glob(os.path.join(pth,'*.txt')):
    name=x.split('.')[0]
    cmd = ['wb_command', '-cifti-label-import', '%s'%(os.path.join(pth,'melodic_IC_ftb_copy.dlabel.nii')), '%s'%(os.path.join(pth,'%s'%x)),'%s'%(os.path.join(pth, '%s.dlabel.nii'%name)), '-discard-others']
    process=subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    process.wait()

## Get ICnames

In [ ]:
g = subgraph_dict['ob'][6]
new={}
for x,y in nx.get_node_attributes(g, 'labels').items():
        new[x]=[y['IC'], y['area']]